Рекуррентные блоки GRU. Пример их реализации в задаче сентимент-анализа | #25 нейросети на Python


In [11]:

import numpy as np
import os
from tensorflow.keras.layers import Dense, SimpleRNN, Input, Embedding,LSTM, GRU, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [12]:
with open('train_data_true', 'r', encoding='utf-8') as f:
    text_true = f.read() 
    text_true = text_true.replace('\ufeff', '')  

In [13]:
with open('train_data_false', 'r', encoding='utf-8') as f:
    text_false = f.read() 
    text_false = text_false.replace('\ufeff', '')  

In [14]:
texts = text_true.splitlines() + text_false.splitlines()
tokenizer.fit_on_texts(texts)
count_true = len(text_true.splitlines())
count_false = len(text_false.splitlines())
total_lines = count_true + count_false
print(count_false, count_true, total_lines)
data = tokenizer.texts_to_sequences(texts)
print(data[:5])  # Первые 5 последовательностей



4102 6598 10700


In [15]:
maxWordsCOunt = 10000
tokenizer = Tokenizer(num_words=maxWordsCOunt, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»', lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts(texts)

In [16]:
dist = list(tokenizer.word_counts.items())
print(dist[:10])
print(texts[0][:100])

[('д', 260), ('у', 231), ('м', 281), ('а', 550), ('й', 125), ('т', 696), ('е', 819), ('п', 241), ('о', 931), ('з', 163)]
Д


In [37]:
max_text_len = 5
data = tokenizer.texts_to_sequences(texts)
data_pad = pad_sequences(data, maxlen=max_text_len)
print(data_pad)
print(list(tokenizer.word_index.items()))

[[ 0  0  0  0 12]
 [ 0  0  0  0 15]
 [ 0  0  0  0 11]
 ...
 [ 0  0  0  0  4]
 [ 0  0  0  0  3]
 [ 0  0  0  0 13]]
[('о', 1), ('е', 2), ('т', 3), ('и', 4), ('н', 5), ('а', 6), ('с', 7), ('в', 8), ('л', 9), ('р', 10), ('м', 11), ('д', 12), ('ь', 13), ('п', 14), ('у', 15), ('к', 16), ('ы', 17), ('я', 18), ('б', 19), ('з', 20), ('ч', 21), ('г', 22), ('й', 23), ('ж', 24), ('ш', 25), ('х', 26), ('ю', 27), ('э', 28), ('щ', 29), ('ц', 30), ('ф', 31), ('ъ', 32), ('ё', 33), ('3', 34)]


In [38]:
x = data_pad
y = np.array([[1, 0]]*count_true + [[0,1]]*count_false)
print(x.shape, y.shape)

(10700, 5) (10700, 2)


In [39]:
indeces = np.random.choice(x.shape[0], size=x.shape[0], replace=False)
x = x[indeces]
y = y[indeces]

In [40]:
model = Sequential()
model.add(Embedding(maxWordsCOunt, 128, input_length=max_text_len))
model.add(GRU(128, return_sequences=True))
model.add(GRU(64))
model.add(Dense(2, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer=Adam(0.01), metrics=['accuracy'])

history = model.fit(x, y, batch_size=32, epochs=5)

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_8 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_9 (GRU)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
335/335 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - accuracy: 0.5917 - loss: 0.6762
Epoch 2/5
335/335 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.6121 - loss: 0.6649
Epoch 3/5
335/335 ━━━━━━━━━━━━━━━━━━━━ 6s 17ms/step - accuracy: 0.6194 - loss: 0.6650
Epoch 4/5
335/335 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.6123 - loss: 0.6674
Epoch 5/5
335/335 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.6204 - loss: 0.6639


In [41]:
def sequence_to_text(list_of_indices):
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

t = 'Я никому не доверяю'.lower()
data = tokenizer.texts_to_sequences([t])
data_pad = pad_sequences(data, maxlen=max_text_len)
print(sequence_to_text(data_pad[0]))

res = model.predict(data_pad)
print(res, np.argmax(res), sep='\n')

[None, None, None, None, 'я']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
[[0.5639057  0.43609428]]
0
